<center><img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/Logo_de_l%27Universit%C3%A9_Jean_Monnet_Saint-Etienne.png" style="height:120px"></center>

<br>
<center><span style="font-size:35px"><b>Advanced Machine Learning</b></span></center>


<center><span style="font-size:25px"><b>Introduction to Adversarial ML</b></span></center>

With recent technological advances, the use of deep neural networks (DNN) have widespread to numerous applications ranging from biomedical imaging to the design of autonomous vehicle. The reasons of their prosperity strongly rely on the increasingly large datasets becoming available, their high expressiveness and their empirical successes in various tasks (e.g. computer vision, natural language processing or speech recognition).

However, their high representation power is also a weakness that some adversary might exploit to craft *adversarial attacks* which could potentially lead the DNN model to take unwanted actions. In order to mitigate the risks of being attack (or, a minima, of misclassifying examples), it is thus necessary to implement *defense mechanisms*. 

Before introducing adversarial attacks and defense mechanisms, we report a quick reminder about classification based deep neural networks.


<br>
<center><img src="https://jordan-frecon.com/images/posts/neuralnet.png" style="height:200px"></center>

Let some dataset $\mathcal{D}=\{x_i,y_i\}_{i=1}^n$ made of $n$ samples $x_i\in\mathcal{X}\subseteq\mathbb{R}^d$ and $y_i\in\mathbb{R}^c$. In addition, let some neural network $f\colon\mathcal{X}\to \mathbb{R}^c$ mapping each input $x\in\mathcal{X}$ to its probabilities $f(x)\in\mathbb{R}^c$ to belong to each of the $c$ classes. Then, the usual way to train $f$ on $\mathcal{D}$ consists in solving

$$
\underset{f}{\text{minimize}} \sum_{i=1}^n H(f(x_i),y_i)
$$

where $H$ is some loss function, typically the cross-entropy. There, the minimization over $f$ is intended over the parameters (weights and biases) of the neural network $f$. Once $f$ is properly trained, the predicted label of any input $x\in\mathcal{X}$ by $f$ is denoted as

$$
C_f(x) = \underset{i\in\{1,\ldots,c\}}{\mathrm{argmax}}\, f_i(x).
$$

### Requirements

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import os
from utils import *
import pytorch_lightning as pl #To install!

# 1. Adversarial Attacks

Although DNN have shown a great success to predict various complex tasks, some concerns have been raised about their safety and more particularly for the safety of the user since the pioneer work of [Szegedy et al., 2014] which has shown the existence of adversarial attacks. The most striking example is probably that of automated vehicles, where malicious attacks could lead the car to take unwanted action with dramatic consequences.

More precisely, adversarial attacks are almost imperceptible transformations aiming to modify an example well classified by a DNN into a new example, called adversarial, which is itself wrongly classified.

<br>
<center><img src="https://jordan-frecon.com/images/posts/adversarial.png" style="height:200px"></center>

Hereafter, we only consider a particular case of adversarial attacks, namely maximum allowable perturbations. More formally, for every image $x$, the adversarial image $a$ will be built as
$$a = x + \varepsilon\quad\text{where}\quad \|\varepsilon\|_p\leq \delta$$
for some maximum budget $\delta>0$. There, the constraint $\|\varepsilon\|_p\leq \delta$ ensures that the adversarial image $a$ is close to the original image $x$, that is $\|a-x\|_p=\|\varepsilon\|_p\leq \delta$. In practice, we use $\delta=1/2$ for $\ell_2$-attacks and $\delta=8/255$ for $\ell_\infty$-attacks.


In order to learn the adversarial perturbation $\varepsilon$ associated to some new example $(x^\prime,y^\prime)$, most methods try to solve:

$$\underset{\varepsilon\in\mathbb{R}^d}{\text{maximize}}\, H(f(x^\prime+\varepsilon),y^\prime)\quad\text{subject to}\quad \|\varepsilon\|_p\leq \delta$$

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Explain and comment the previous optimization problem in your own words. What can you say in terms of 1) modeling and 2) difficulty to solve?

In what follows, you will learn a couple of strategies to craft adversarial attacks. Every attack will necessarily face a trade-off between some or all the following performance criteria

> **Performance criteria.**
> Let some set $\{x_i,a_i\}_{i=1}^n$ made of $n$ instances $x_i$ and their adversarial examples $a_i$ crafted by means of some attack strategy on the neural network $f$. In order to judge upon the quality of the attack, the most common criteria are the following.
> - *Fooling rate:* the fraction of adversarial examples which do fool the classifier, i.e., $\frac{1}{n} \sum_{i=1}^n \mathbb{1}_{C_f(x_i)\neq C_f(a_i)}$.
>- *Computational complexity:* Cost of the algorithm used to craft adversarial examples.
> - $\ell_p$*-budget:* the amount of distortion/perturbation measured in terms of mean $\ell_p$-norm, i.e., $\frac{1}{n}\sum_{i=1}^n \|a_i-x_i\|_p$.
> - *Transferability:* the fooling rate obtained on a neural network $f^\prime$ different from $f$ on which the attack is learned.

## 1.1. Simple attack on MNIST

### 1.1.1. Preparing the dataset and model

We consider the MNIST dataset extracted by resorting to the *torchvision* package. In particular, we only use the test set (see `train=False`) since the training set is typically used to train models.

In [ ]:
import torchvision.transforms as transforms

mnist = torchvision.datasets.MNIST

data_transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(affine_transformation)
                                ])

test_set = mnist(root='data', train=False, download=True, transform=data_transform)

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Explain the role of `transform=data_transform`. What does it perform?

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Visualize the first example from the test set. If you struggle, you can use the function *display_example* from the *utils.py* file.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Create an instance of the multi-layer perceptron (MLP) called *CiresanMLP*, with `network_id=5`, from the *utils.py* file. Then, load the parameters of the pretrained models from the file *models/MNIST/model_ciresan_mnist_id5.pt*. To this purpose, refer to the tutorial https://pytorch.org/tutorials/beginner/saving_loading_models.html. Make sure to use  the keyword `map_location=torch.device('cpu')`.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Create a PyTorch dataloader of the test set with batch size 10. Verify that the test accuracy is about $98.52\%$.

### 1.1.2. One-shot attack: FGSM

The *Fast Gradient Sign Method* (FGSM) [Goodfellow et al., 2015] is one of the first effective technique to craft an $\ell_\infty$-adversarial perturbation. The underlying idea is to define the perturbation $\varepsilon$ as $\delta$ multiplied by the sign of the gradient of the training loss with respect to the input image $x^\prime$. Mathematically:
$$\varepsilon =  \delta \; \mathrm{sign}\left( \nabla_x H(f(x^\prime),y^\prime) \right).$$

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Why does the FGSM attack ensures that the adversarial perturbation lies in a $\ell_\infty$-ball of radius $\delta$ ?

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Compute the FGSM perturbation associated to the first example of the test set

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Create a copy (named *display_attack*) of the function *display_example* from the *utils.py* in order to show: 1) the original image and its predicted label, 2) the adversarial perturbation, and 3) the adversarial example with its predicted label.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Take a look at the first 10 images of the test, their corresponding FGSM perturbation, and their associated aversarial image. What do you observe? Comment.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  What do you observe ? Does the adversarial image $a=x+\varepsilon$ always belongs to $$\mathcal{X}=\{\text{set of images with pixels' value between 0 and 1}\}$$ Fix the algorithmic solution accordingly.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Evaluate the quality of the FGSM attacks in terms of *Fooling rate*, that is the fraction of adversarial examples which do fool the classifier, i.e., $$\frac{1}{n} \sum_{i=1}^n \mathbb{1}_{C_f(x_i)\neq C_f(a_i)}.$$

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Devise a variant of FGSM to craft $\ell_2$-constrained adversarial perturbations. Evaluate and comment its fooling rate.

## 1.2. Advanced Attacks on CIFAR-10

### 1.2.1. Preparing the dataset and model

We start by extracting the test set of CIFAR-10 to the data folder and preprocessing the data to transform all images into tensors.

In [ ]:
cifar10 = torchvision.datasets.CIFAR10
test_set = cifar10(root='data', train=False, download=True, transform=transforms.ToTensor())

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Display one image as well as its corresponding label. You might be required to permutate the dimensions of the image from `x` (RGB, Height, Width) to `np.transpose(x, (1, 2, 0))` (Height, Width, RGB) in order to display it.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** We are going to use multiple pretrained models on CIFAR-10. To this purpose, first read the documentation README.md inside the folder *models/CIFAR10* in order to download the weights of the networks. You will be asked to execute the following command-line:<br>
`python train.py --download_weights 1`<br>
Make sure that you are in an environment with the *pytorch_lightning* package installed!

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Load a pretrained ResNet18 model by using the following commands. Why do we add *Normalize()*? Why is it necessary to add `.eval()` at the end?

```
from models.CIFAR10.cifar10_models.resnet import resnet18
model = torch.nn.Sequential(
    Normalize(),
    resnet18(pretrained=True).eval().to(device=device)
)
model = model.eval()
```

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Make sure the ResNet18 model has been successfully loaded by checking its test accuracy.

### 1.2.2. Attacks on pre-trained CIFAR-10 classifiers

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Evaluate the performance of FGSM attacks on the previous ResNet18 model

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** We now consider a new attack called *PGD*. Given some learning rate $\mu>0$, it reads

$$
\begin{align}
 &\text{initialize }\varepsilon\\
    &\text{for } k=1\ldots K\\[0.4ex]
    &\left\lfloor\begin{array}{l}
    \varepsilon \leftarrow \mathrm{Proj}_{\|\cdot\|_{p}\leq\delta} \left( \varepsilon + \mu \nabla_\varepsilon H(f(x^\prime+\varepsilon),y^\prime) \right)\\
    \end{array}\right.
\end{align}
$$

Can you guess how to smartly choose $\mu$ and initialize $\varepsilon$ in order to ensure that the perturbation always lies inside the $\ell_p$-ball ?

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Implement the corresponding algorithmic solution

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Evaluate the performance of PGD attacks on the previous ResNet18 model

### 1.2.3. Transferability of attacks

We are now ready to investigate if the FGSM and PGD attacks learned on the ResNet18 model are also able to fool other networks.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Evaluate the transferability of FGSM, learned on $f$ being the ResNet18, to 3 other networks $f^\prime$ from the  *models/CIFAR10* folder

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Same question for the PGD attack.

### 1.2.4. Advanced attacks

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Take a look at the literature and explain, in a few words, a new attack.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Implement the attack by yourself or use some PyTorch implementation (cf. [TorchAttack](https://github.com/Harry24k/adversarial-attacks-pytorch))

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.** Evaluate its transferability on various CIFAR-10 classifiers

# 2. Adversarial Defense

Usually, the parameters of a deep neural network (DNN) $f$ are trained by minimizing the following training loss, i.e.,
$$
\underset{f}{\text{minimize}} \sum_{i=1}^n H(f(x_i),y_i)
$$
However, as previously discussed, the resulting trained DNN is vulnerable to adversarial attacks able to fool the DNN.

To mitigate the impact of adversarial examples, early works have suggested to train DNN on mixtures of clean and adverarial examples, thus leading to *Adversarial Training* techniques, that will be studied here. Note that, departing from these techniques, one could also consider a more theoretically grounded framework, called *Adversarial Robust Training*, or noise-injection mechanisms to induce smoothness, regularization techniques and various other strategies. However, all theses techniques are beyond the scope of this exercice.


The idea of adversarial training relies in also training the neural network model on adversarial examples in order to correctly predict their label.

> Such type of techniques is also commonly called *Adversarial Retraining* in order to further contrast with the methods discussed in the next section.

The work of [Szegedy et al., 2014] first showed that by training a neural network $f$ on a mixture of clean and adversarial examples, then $f$ can be somewhat robustified. The corresponding training procedures amounts in solving

$$
\underset{\theta\in\Theta}{\mathrm{minimize}}\; \sum_{i=1}^n \Big( \alpha H(f(x_i),y_i) + (1-\alpha) H(f(x_i+\varepsilon_i),y_i) \Big)
$$

where $\alpha\in]0,1[$ and $\varepsilon_i$ denotes an adversarial perturbation crafted from $x$.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Define a simple MLP model to perform digits prediction on MNIST

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Implement the above-mentionned *adversarial retraining* strategy in order to learn a robust model. You can use any attack you want in order to learn the adversarial perturbations.

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Find the best value of the coefficient $\alpha$

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Evaluate the accuracy of your model

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Evaluate the fooling rates of FGSM and PGD attacks on your model

 **<i class="fa fa-gear fa-spin fa-1x"></i> EXERCISE.**  Compare with a non-robust version of your model. Comment

## References

- [Szegedy et al., 2014]  C. Szegedy, W. Zaremba, I. Sutskever, J. Bruna, D. Erhan, I. Goodfellow and R. Fergus. "Intriguing properties of neural networks". International Conference on Learning Representations (ICLR) (2014)
- [Goodfellow et al., 2015]  I. Goodfellow, J. Shlens and C. Szegedy. "Explaining and Harnessing Adversarial Examples". International Conference on Learning Representations (ICLR) (2015)
